In [1]:
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os

from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
class Create1DConvCoreModel(nn.Module):
    def __init__(self, input_shape):
        super(Create1DConvCoreModel, self).__init__()

        self.conv1d_1 = nn.Conv1d(in_channels=input_shape[1], out_channels=32, kernel_size=24)
        self.relu_1 = nn.ReLU()
        self.conv1d_2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=16)
        self.relu_2 = nn.ReLU()
        self.conv1d_3 = nn.Conv1d(in_channels=64, out_channels=96, kernel_size=8)
        self.relu_3 = nn.ReLU()
        self.maxpool_1d = nn.MaxPool1d(kernel_size=20)
        self.linear_1 = nn.Linear(in_features=7*96, out_features=256)
        self.relu_4 = nn.ReLU()
        self.linear_2 = nn.Linear(in_features=256, out_features=5)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1d_1(x)
        x = self.relu_1(x)
        x = self.conv1d_2(x)
        x = self.relu_2(x)
        x = self.conv1d_3(x)
        x = self.relu_3(x)
        x = self.maxpool_1d(x)
        x = x.view(x.size(0), -1)
        x = self.linear_1(x)
        x = self.relu_4(x)
        x = self.linear_2(x)
        x = self.softmax(x)

        return x

In [3]:
data_dir = '/media/darshana/Software/dataset'
input_shape = (200, 3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
label_encoder = LabelEncoder()

# Create an instance of the model and move it to the GPU device
model = Create1DConvCoreModel(input_shape).to(device)

num_epochs = 1
batch_size = 256

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# List of subjects to be used for training
subjects_train = [f'P{str(i).zfill(3)}.obj' for i in range(1, 8)]

# Train the model incrementally using each subject's data
for subject_idx, subject in enumerate(subjects_train):
    with open(os.path.join(data_dir, subject), 'rb') as f:
        data = pickle.load(f)
    print(f'Loaded data for {subject}')

    data['y'] = label_encoder.fit_transform(data['y'])
    print(f'Encoded labels for {subject}')

    # Load the saved model if this is not the first subject
    if subject_idx > 0:
        model.load_state_dict(torch.load(f'model_after_{subjects_train[subject_idx - 1][:-4]}.pth'))

    # Train the model using the data from the pickle file
    # Assuming data['X'] and data['y'] contain the input data and labels
    for epoch in range(num_epochs):

        # Create a DataLoader to iterate over the data in batches
        dataset = TensorDataset(torch.tensor(data['X'], dtype=torch.float32), 
                                torch.tensor(data['y'], dtype=torch.long))
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        for i, (x, y) in enumerate(dataloader):
            # Move x and y to the GPU device
            x = x.to(device)
            y = y.to(device)
            x = torch.transpose(x, 1, 2)

            # Run the model forward
            y_pred = model(x)

            # Compute the loss
            loss = criterion(y_pred, y)

            # Compute the gradients and update the model
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i%100 == 0:
                print (f'Subject: {subject}; Turn: {i//100}; Loss: {loss.item():.4f}')

    # Save the model after training with the current subject's data
    torch.save(model.state_dict(), f'model_after_{subject[:-4]}.pth')

    # Clear memory
    del data
    torch.cuda.empty_cache()

Loaded data for P001.obj
Encoded labels for P001.obj
Subject: P001.obj; Turn: 0; Loss: 1.6091
Subject: P001.obj; Turn: 1; Loss: 1.4335
Subject: P001.obj; Turn: 2; Loss: 1.4739
Subject: P001.obj; Turn: 3; Loss: 1.4429
Subject: P001.obj; Turn: 4; Loss: 1.4499
Subject: P001.obj; Turn: 5; Loss: 1.4346
Subject: P001.obj; Turn: 6; Loss: 1.4562
Subject: P001.obj; Turn: 7; Loss: 1.4281
Subject: P001.obj; Turn: 8; Loss: 1.4349
Subject: P001.obj; Turn: 9; Loss: 1.4588
Subject: P001.obj; Turn: 10; Loss: 1.4411
Subject: P001.obj; Turn: 11; Loss: 1.4690
Subject: P001.obj; Turn: 12; Loss: 1.4413
Subject: P001.obj; Turn: 13; Loss: 1.4289
Subject: P001.obj; Turn: 14; Loss: 1.4283
Subject: P001.obj; Turn: 15; Loss: 1.4495
Subject: P001.obj; Turn: 16; Loss: 1.4402
Subject: P001.obj; Turn: 17; Loss: 1.4650
Subject: P001.obj; Turn: 18; Loss: 1.4669
Subject: P001.obj; Turn: 19; Loss: 1.4613
Loaded data for P002.obj
Encoded labels for P002.obj
Subject: P002.obj; Turn: 0; Loss: 1.6802
Subject: P002.obj; Turn

KeyboardInterrupt: 

In [ ]:
# List of subjects to be used for testing
subjects_test = [f'P{str(i).zfill(3)}.obj' for i in range(8, 11)]

# Test the model using each subject's data
for subject in subjects_test:
    with open(os.path.join(data_dir, subject), 'rb') as f:
        data = pickle.load(f)

    data['y'] = label_encoder.fit_transform(data['y'])

    # Create a DataLoader to iterate over the test data in batches
    test_dataset = TensorDataset(torch.tensor(data['X'], dtype=torch.float32), 
                                torch.tensor(data['y'], dtype=torch.long))
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    correct_t = 0
    total_t = 0

    # Iterate over the test data in batches
    with torch.no_grad():
        for x, y in test_dataloader:
            # Move x to the GPU device and transpose it
            x = x.to(device)
            x = torch.transpose(x, 1, 2)
            
            # Run the model forward
            test_outputs = model(x)
            
            # Compute the predicted and actual labels
            _, predictedt = torch.max(test_outputs, 1)
            actual = y
            
            # Update the correct and total counts
            correct_t += (predictedt.cpu() == actual).sum().item()
            total_t += y.size(0)

    # Calculate and print test accuracy
    test_accuracy = correct_t / total_t * 100
    print('Test accuracy:')
    print(test_accuracy)

    # Clear memory
    del data
    torch.cuda.empty_cache()


Test accuracy:
38.3515525718734


In [ ]:
# # Define the path to the directory containing the pickle files
# data_dir = '/content/drive/Shareddrives/GitRepos/fyp-phase-2/preprocessing'

# # Define the input shape of the model
# input_shape = (200, 3)

# # Check if GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Loop over the pickle files
# for user_id in range(1, 11):
#     # Load the pickle file for the current user
#     with open(data_dir + f'/P{user_id:03d}.obj', 'rb') as f:
#         data = pickle.load(f)

#     # Create an instance of the model and move it to the GPU device
#     model = Create1DConvCoreModel(input_shape).to(device)

#     num_epochs = 10

#     criterion = nn.CrossEntropyLoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#     label_encoder = LabelEncoder()
#     data['y'] = label_encoder.fit_transform(data['y'])

#     # Train the model using the data from the pickle file
#     # Assuming data['X'] and data['y'] contain the input data and labels
#     for epoch in range(num_epochs):
#         epoch_loss = 0
#         epoch_acc = 0
#         epoch_f1 = 0
#         epoch_r2 = 0

#         for i, (x, y) in enumerate(zip(data['X'], data['y'])):
#             # Convert x and y to tensors and move them to the GPU device
#             x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(0).to(device)
#             y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(0).to(device)

#             print(x_tensor[1])

#             # Run the model forward
#             y_pred = model(x_tensor)

#             # Compute the loss
#             loss = criterion(y_pred, y_tensor)

#             # Compute the gradients and update the model
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             # Compute the accuracy, F1 score, and R2 score for the current batch
#             y_pred = torch.argmax(y_pred, dim=1)
#             acc = accuracy_score(y_tensor.cpu(), y_pred.cpu())
#             f1 = f1_score(y_tensor.cpu(), y_pred.cpu(), average='weighted')
#             r2 = r2_score(y_tensor.cpu(), y_pred.cpu())

#             # Accumulate the metrics for the current epoch
#             epoch_loss += loss.item()
#             epoch_acc += acc
#             epoch_f1 += f1
#             epoch_r2 += r2

#         # Compute the average metrics for the epoch
#         num_batches = len(data['X'])
#         epoch_loss /= num_batches
#         epoch_acc /= num_batches
#         epoch_f1 /= num_batches
#         epoch_r2 /= num_batches

#         # Print the metrics for the epoch
#         print(f'User {user_id}, Epoch {epoch+1}:')
#         print(f'  Loss: {epoch_loss:.4f}')
#         print(f'  Accuracy: {epoch_acc:.4f}')
#         print(f'  F1 Score: {epoch_f1:.4f}')
#         print(f'  R2 Score: {epoch_r2:.4f}')

#     # Delete the data object to free up memory
#     del data